In [86]:
import cv2
import glob
import keras.models
import numpy as np
from tabulate import tabulate

In [2]:
pwd

u'/Users/PaulYim/Desktop/car/computer/notebooks'

In [6]:
model = keras.models.load_model('../nn_h5/nn.h5')
model

In [87]:
def auto_canny(blurred):
    # Compute the median of the single channel pixel intensities
    sigma = 0.33
    v = np.median(blurred)

    # Apply automatic Canny edge detection using the computed median
    lower = int(max(0,   (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(blurred, lower, upper)

    # Return the edged image
    return edged

In [88]:
def preprocess_for_predict(filepath_image):
    image = cv2.imread(filepath_image, cv2.IMREAD_GRAYSCALE)
    roi = image[120:240, :]
    blurred = cv2.GaussianBlur(roi, (3, 3), 0)
    auto = auto_canny(blurred)
    image_array = auto.reshape(1, 38400).astype(np.float32)
    image_array = image_array/255.
    return image_array

In [92]:
'''One-image-at-a-time prediction test'''

# '''Forward'''
# filepath_image = '../images/imgs_20161024_201843/frame00000.jpg'

'''Forward-Right'''
filepath_image = '../images/imgs_20161024_201843/frame00020.jpg'

# '''Forward-Left'''
# filepath_image = '../images/imgs_20161024_203956/frame00055.jpg'

image_array = preprocess_for_predict(filepath_image)
y_hat = model.predict(image_array)
print y_hat
i_max       = np.argmax(y_hat)
y_hat_final = np.zeros((1,3))
np.put(y_hat_final, i_max, 1)
print y_hat_final

[[  1.44709367e-04   9.10869002e-01   8.89862105e-02]]
[[ 0.  1.  0.]]


In [99]:
'''Prediction on a brand new test set (Fri, Oct 28)'''

filepath_images = '../test_set_2016_10_28/*.jpg'
filepath_labels = '../test_set_2016_10_28/label_array_ORIGINALS.npz'

def predict_testset(filepath_images):
    model = keras.models.load_model('../nn_h5/nn.h5')
    images      = glob.glob(filepath_images)
    labels_file = glob.glob(filepath_labels)
    num_predictions = len(images)
    correct = 0
    
    # Extract y values (actual labels)
    for single_npz in labels_file:
        with np.load(single_npz) as data:

            # ORIGINAL LABELS (y values)
            labels = data.f.train_labels
    
    # Predict on each image
    predictions        = []
    predictions_argmax = []

    for each in images:
        image_array = preprocess_for_predict(each)
        y_hat       = model.predict(image_array)
        i_max       = np.argmax(y_hat)
        y_hat_final = np.zeros((1,3))
        np.put(y_hat_final, i_max, 1)
        y_hat = np.around(y_hat, decimals=3)    
        predictions.append(y_hat)
        predictions_argmax.append(y_hat_final)
        
    # Tabulate all results
    table = []
    for i,j in enumerate(xrange(num_predictions)):
        if np.all(predictions_argmax[i]==labels[i]):
            correct += 1
        
        row = [i, \
               predictions[i][0], \
               predictions_argmax[i][0], \
               labels[i], \
               "" if np.all(predictions_argmax[i]==labels[i]) == 1 else "Wrong"]
        
        table.append(row)
    
    print 'Number of correct predictions:', correct
    print 'Number of incorrect predictions:', num_predictions-correct
    print 'Accuracy:', float(correct)/num_predictions
    print ''
    print tabulate(table, headers=['Frame', 'Prediction_prob', 'Prediction_argmax', 'Actual', 'Wrongness'])
    
predict_testset(filepath_images)

Number of correct predictions: 149
Number of incorrect predictions: 38
Accuracy: 0.79679144385

  Frame  Prediction_prob                        Prediction_argmax    Actual         Wrongness
-------  -------------------------------------  -------------------  -------------  -----------
      0  [ 0.007       0.009       0.98500001]  [ 0.  0.  1.]        [ 0.  0.  1.]
      1  [ 0.          0.004       0.99599999]  [ 0.  0.  1.]        [ 0.  0.  1.]
      2  [ 0.001       0.003       0.99699998]  [ 0.  0.  1.]        [ 0.  0.  1.]
      3  [ 0.  0.  1.]                          [ 0.  0.  1.]        [ 0.  0.  1.]
      4  [ 0.  0.  1.]                          [ 0.  0.  1.]        [ 0.  0.  1.]
      5  [ 0.  0.  1.]                          [ 0.  0.  1.]        [ 0.  0.  1.]
      6  [ 0.002       0.          0.99800003]  [ 0.  0.  1.]        [ 0.  0.  1.]
      7  [ 0.002       0.          0.99800003]  [ 0.  0.  1.]        [ 0.  0.  1.]
      8  [ 0.001       0.          0.99800003]  [ 